# Clone official repo

In [ ]:
! git clone https://github.com/xuebinqin/DIS
%cd ./DIS/IS-Net
!pip install gdown

Cloning into 'DIS'...
remote: Enumerating objects: 343, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 343 (delta 89), reused 66 (delta 66), pack-reused 234
Receiving objects: 100% (343/343), 49.63 MiB | 32.74 MiB/s, done.
Resolving deltas: 100% (165/165), done.
/content/DIS/IS-Net


# Imports

In [ ]:
import numpy as np
from PIL import Image
import torch
from torch.autograd import Variable
from torchvision import transforms
import torch.nn.functional as F
import gdown
import os
from google.colab.patches import cv2_imshow
import cv2

import requests
import matplotlib.pyplot as plt
from io import BytesIO

# project imports
from data_loader_cache import normalize, im_reader, im_preprocess
from models import *
from pathlib import Path
from tqdm import tqdm


# Helpers

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Download official weights
if not os.path.exists("./saved_models"):
    !mkdir ./saved_models
    MODEL_PATH_URL = "https://drive.google.com/uc?id=1KyMpRjewZdyYfxHPYcd-ZbanIXtin0Sn"
    gdown.download(MODEL_PATH_URL, "./saved_models/isnet.pth", use_cookies=False)


class GOSNormalize(object):
    '''
    Normalize the Image using torch.transforms
    '''
    def __init__(self, mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]):
        self.mean = mean
        self.std = std

    def __call__(self,image):
        image = normalize(image,self.mean,self.std)
        return image


transform =  transforms.Compose([GOSNormalize([0.5,0.5,0.5],[1.0,1.0,1.0])])

def load_image(im_path, hypar):
    if im_path.startswith("http"):
        im_path = BytesIO(requests.get(im_path).content)

    im = im_reader(im_path)
    im, im_shp = im_preprocess(im, hypar["cache_size"])
    im = torch.divide(im,255.0)
    shape = torch.from_numpy(np.array(im_shp))
    return transform(im).unsqueeze(0), shape.unsqueeze(0) # make a batch of image, shape


def build_model(hypar,device):
    net = hypar["model"]#GOSNETINC(3,1)

    # convert to half precision
    if(hypar["model_digit"]=="half"):
        net.half()
        for layer in net.modules():
            if isinstance(layer, nn.BatchNorm2d):
                layer.float()

    net.to(device)

    if(hypar["restore_model"]!=""):
        net.load_state_dict(torch.load(hypar["model_path"]+"/"+hypar["restore_model"],map_location=device))
        net.to(device)
    net.eval()
    return net


def predict(net,  inputs_val, shapes_val, hypar, device):
    '''
    Given an Image, predict the mask
    '''
    net.eval()

    if(hypar["model_digit"]=="full"):
        inputs_val = inputs_val.type(torch.FloatTensor)
    else:
        inputs_val = inputs_val.type(torch.HalfTensor)


    inputs_val_v = Variable(inputs_val, requires_grad=False).to(device) # wrap inputs in Variable

    ds_val = net(inputs_val_v)[0] # list of 6 results

    pred_val = ds_val[0][0,:,:,:] # B x 1 x H x W    # we want the first one which is the most accurate prediction

    ## recover the prediction spatial size to the orignal image size
    pred_val = torch.squeeze(F.upsample(torch.unsqueeze(pred_val,0),(shapes_val[0][0],shapes_val[0][1]),mode='bilinear'))

    ma = torch.max(pred_val)
    mi = torch.min(pred_val)
    pred_val = (pred_val-mi)/(ma-mi) # max = 1

    if device == 'cuda': torch.cuda.empty_cache()
    return (pred_val.detach().cpu().numpy()*255).astype(np.uint8) # it is the mask we need

Downloading...
From: https://drive.google.com/uc?id=1KyMpRjewZdyYfxHPYcd-ZbanIXtin0Sn
To: /content/DIS/IS-Net/DIS/IS-Net/saved_models/isnet.pth
100%|██████████| 177M/177M [00:02<00:00, 86.1MB/s]


# Set Parameters

In [ ]:
#when using google colab, run this code
"""
from google.colab import drive
drive.mount('/content/drive')
"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
hypar = {} # paramters for inferencing


hypar["model_path"] ="./saved_models" ## load trained weights from this path
hypar["restore_model"] = "isnet.pth" ## name of the to-be-loaded weights
hypar["interm_sup"] = False ## indicate if activate intermediate feature supervision

##  choose floating point accuracy --
hypar["model_digit"] = "full" ## indicates "half" or "full" accuracy of float number
hypar["seed"] = 0

hypar["cache_size"] = [1024, 1024] ## cached input spatial resolution, can be configured into different size

## data augmentation parameters ---
hypar["input_size"] = [1024, 1024] ## mdoel input spatial size, usually use the same value hypar["cache_size"], which means we don't further resize the images
hypar["crop_size"] = [1024, 1024] ## random crop size from the input, it is usually set as smaller than hypar["cache_size"], e.g., [920,920] for data augmentation

hypar["model"] = ISNetDIS()

# Build Model

In [ ]:
net = build_model(hypar, device)

# Predict Mask

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
import gradio as gr

def inference(image):
  image_path = image

  image_tensor, orig_size = load_image(image_path, hypar)
  mask = predict(net, image_tensor, orig_size, hypar, device)

  pil_mask = Image.fromarray(mask).convert('L')
  im_rgb = Image.open(image).convert("RGB")

  im_rgba = im_rgb.copy()
  im_rgba.putalpha(pil_mask)

  return [im_rgba, pil_mask]

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:3737: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [ ]:
#experiment with normal images

def inference(image):
  image_path = image

  image_tensor, orig_size = load_image(image_path, hypar)
  mask = predict(net, image_tensor, orig_size, hypar, device)

  pil_mask = Image.fromarray(mask).convert('L')
  im_rgb = Image.open(image).convert("RGB")

  im_rgba = im_rgb.copy()
  im_rgba.putalpha(pil_mask)

  return [im_rgba, pil_mask]

#inference images in the folder
masked_images=[]
input_folder = '/content/drive/MyDrive/Colab Notebooks/smoretalk/input_image_normal'#path 지정
output_folder = '/content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal'#path 지정

# import required module
import glob
masked_images = []

for img in tqdm(sorted(glob.glob(os.path.join(input_folder, "/*.png")))): #들어가는 이미지의 확장자가 png
    masked_images.append(inference(img)[0])



#save outputs in output_folder
#output_folder.save(masked_images)


  0%|          | 0/58 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:3737: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


/content/drive/MyDrive/Colab Notebooks/smoretalk/input_image_normal/20061.png


  2%|▏         | 1/58 [00:04<04:37,  4.87s/it]

/content/drive/MyDrive/Colab Notebooks/smoretalk/input_image_normal/20153.png


  3%|▎         | 2/58 [00:10<04:45,  5.11s/it]

...

 97%|█████████▋| 56/58 [04:58<00:10,  5.21s/it]

/content/drive/MyDrive/Colab Notebooks/smoretalk/input_image_normal/24249.png


 98%|█████████▊| 57/58 [05:03<00:05,  5.15s/it]

/content/drive/MyDrive/Colab Notebooks/smoretalk/input_image_normal/24270.png


100%|██████████| 58/58 [05:08<00:00,  5.32s/it]


In [ ]:
input_folder =  '/content/drive/MyDrive/Colab Notebooks/smoretalk/input_image_normal'#path 지정
output_folder = '/content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal'#path 지정


In [ ]:
import os

# Assuming you have a list of file paths for the images in another directory
other_directory_path = input_folder
other_directory_images = sorted(os.listdir(other_directory_path))

# Assuming you have a list of processed images in masked_images
for i, img in enumerate(masked_images):
    if i < len(other_directory_images):
        # Get the name of the image in the other directory
        image_name = other_directory_images[i]

        # Define the output directory where you want to save the processed images
        output_directory = output_folder

        # Construct the output file path with the same name as the image in the other directory
        output_file_path = os.path.join(output_directory, image_name)

        # Save the processed image with the same name in the output directory
        img.save(output_file_path)

        print(f"Saved processed image {i + 1} as {image_name} in {output_directory}")
    else:
        print(f"No corresponding image found in the other directory for processed image {i + 1}")

Saved processed image 1 as 20061.png in /content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal
Saved processed image 2 as 20153.png in /content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal
Saved processed image 3 as 20245.png in /content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal
Saved processed image 4 as 20335.png in /content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal
Saved processed image 5 as 20394.png in /content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal
Saved processed image 6 as 20436.png in /content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal
Saved processed image 7 as 20479.png in /content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal
Saved processed image 8 as 20510.png in /content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal
Saved processed image 9 as 20568.png in /content/drive/MyDrive/Colab Notebooks/smoretalk/output_image_normal
Saved processed ima